<a href="https://colab.research.google.com/github/Aparna-U-Nair/Simpsons-Character-Recognition/blob/master/SimpsonsCharRecog_LittleVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
from os.path import join, isfile
from os import listdir
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.layers.advanced_activations import ELU

In [ ]:
num_classes = 20
rows,cols = 32,32
batch_size = 32

train_path = "/content/gdrive/My Drive/Colab Notebooks/simpsons/train/"
test_path = "/content/gdrive/My Drive/Colab Notebooks/simpsons/test"

#Using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range =30,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = "nearest",
    rescale = 1./255,
)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(rows,cols),
        batch_size=batch_size,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(rows,cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 19558 images belonging to 20 classes.
Found 990 images belonging to 20 classes.


In [ ]:
#Create the VGG9 model
model = Sequential()

#1st Conv-Relu Layer
model.add(Conv2D(64,(3,3),input_shape = (rows,cols,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#2nd Conv-Relu Layer
model.add(Conv2D(64,(3,3),input_shape = (rows,cols,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#Maxpooling with Dropout Layer
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

#3rd Conv-Relu Layer
model.add(Conv2D(128,(3,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#4th Conv-Relu Layer
model.add(Conv2D(128,(3,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#Maxpooling with Dropout Layer
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

#5th Conv-Relu Layer
model.add(Conv2D(256,(3,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#6th Conv-Relu Layer
model.add(Conv2D(256,(3,3),padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

#Maxpooling with Dropout Layer
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

#1st FC layer
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#2nd FC layer
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Final output layer
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_9 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_10 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)       

In [ ]:
#Model Visualization
%matplotlib inline
from keras.utils.vis_utils import plot_model
from matplotlib.image import mpimg
plot_model(model,to_file = "/content/gdrive/My Drive/Colab Notebooks/simpsons/VGG9.png", show_shapes = True, show_layer_names = True)
img = mpimg.imread("/content/gdrive/My Drive/Colab Notebooks/simpsons/VGG9.png")
plt.figure(figsize=(100,70))
plot = plt.imshow(img)

ImportError: ignored

In [ ]:
#Train the VGG9 model

from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath = "/content/gdrive/My Drive/Colab Notebooks/simpsons/model.{epoch:02d}-{val_loss:.2f}.h5",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1)
es = EarlyStopping(monitor = "val_loss",
                   min_delta = 0,
                   patience = 3,
                   verbose = 1,
                   restore_best_weights = True)
lr = ReduceLROnPlateau(monitor = "val_loss",
                       factor = 0.2,
                       min_delta = 0.00001,
                       patience = 3,
                       verbose = 1)

callbacks = [checkpoint, es, lr]

model.compile(loss = "categorical_crossentropy",
              optimizer = Adam(lr = 0.01),
              metrics = ["accuracy"])

#sample size from data generator out
train_samples = 19548
test_samples = 990
epochs = 20

hist = model.fit_generator(train_generator,
                           steps_per_epoch = train_samples // batch_size,
                           epochs = epochs,
                           callbacks = callbacks,
                           validation_data = test_generator,
                           validation_steps = test_samples//batch_size
                           )

Epoch 1/20
610/610 [==============================] - ETA: 0s - loss: 2.8840 - accuracy: 0.1670
Epoch 00001: val_loss improved from inf to 2.64472, saving model to /content/gdrive/My Drive/Colab Notebooks/simpsons/model.01-2.64.h5
610/610 [==============================] - 5867s 10s/step - loss: 2.8840 - accuracy: 0.1670 - val_loss: 2.6447 - val_accuracy: 0.2115
Epoch 2/20
610/610 [==============================] - ETA: 0s - loss: 2.2980 - accuracy: 0.2926
Epoch 00002: val_loss improved from 2.64472 to 2.55723, saving model to /content/gdrive/My Drive/Colab Notebooks/simpsons/model.02-2.56.h5
610/610 [==============================] - 84s 138ms/step - loss: 2.2980 - accuracy: 0.2926 - val_loss: 2.5572 - val_accuracy: 0.2990
Epoch 3/20
407/610 [===================>..........] - ETA: 24s - loss: 1.9185 - accuracy: 0.4041

In [ ]:
# model.save("/content/gdrive/My Drive/Colab Notebooks/simpsons/last_vgg9.h5")

In [ ]:
#Performance Analysis
from sklearn.metrics import classification_report, confusion_matrix

#Recreating the test datagen by keeping shuffle = False
val_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(rows,cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False)

labels = val_generator.class_indices
labels = {v:k for k,v in labels.items()}
classes= list(labels.values())

train_samples = 19548
test_samples = 990

y_pred = model.predict_generator(val_generator, test_samples//batch_size+1)
y_pred = np.argmax(y_pred,axis = 1)

cm = confusion_matrix(val_generator.classes, y_pred)
print("CM")
cm
print("Class Report")
classification_report(val_generator.classes, y_pred, target_names = classes)

plt.figure(figsize=(8,8))
plt.imshow(cm,interpolation="nearest")
plt.colorbar()
tick_marks = len(classes)
_ = plt.xticks(tick_marks,classes, rotation =90 )
_ = plt.yticks(tick_marks, classes)


In [ ]:
#Testing some random images
def draw_test_img(name,pred,input_img,true_label):
  black = [0,0,0]
  final_img = cv2.copyMakeBorder(input_img,160,0,0,300, cv2.BORDER_CONSTANT, value =BLACK)
  cv2.putText(final_img,"Predicted: "+pred,(20,60),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),2)
  cv2.putText(final_img,"True: "+true_label,(20,120),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),2)
  cv2.imshow(name,final_img)
    
def get_rand_img(path, wid, hei):


width, height = 32,32
files= []
preds = []
true_lab = []

for i in range(5):
  path = "/content/gdrive/My Drive/Colab Notebooks/simpsons/validation/"
  img, fin_path, true_label = get_rand_img(path, width, height)
  files.append(fin_path)
  true_lab.append(true_label)
  x = image.img_to_array(img)
  x = x* 1./255
  x = np.expand_dims(x, axis = 0)
  images= np.vstack([x])
  classes = model.predict_classes(images, batch_size=10)
  preds.append(classes)

for i in range(len(files)):
  image = cv2.imread(files[i])
  image = cv2.resize(image, None, fx = 5, fy = 5, interpolation = cv2.INTER_CUBIC)
  draw_test("Prediction",)
  
  rand_num = np.random.randint(0,len(X_test))
  rand_img = X_test[rand_num]
  img = cv2.resize(rand_img,None, fx=4,fy=4, interpolation=cv2.INTER_CUBIC)
  img = img.reshape(1,X_test[0].shape[0], X_test[1].shape[0],1)
  
  out = str(mnist_model.predict_classes(img,1,verbose=0)[0])
  draw_test_img(out, img)
    
cv2.waitKey(0)
cv2.destroyAllWindows()